# Werewolf Simulator
The following runs a simulation of the game Werewolf with multiple agents, it is built on top of and uses the Agentscope library, documentation can be found here https://doc.agentscope.io/, and the open source code that we extend on is here https://github.com/modelscope/agentscope/tree/main.
### Rules of Werewolf
- There are 4 roles, the Werewolves, Villagers, Witch, and Seer (Witch and Seer are on the Villager team)
- Each night:
    - The Werewolves discuss and vote on a player to eliminate
    - The Witch is told what player the wolves voted to eliminate, and is given the choice to use their potion of healing to reserruct the eliminated player, or use their poison to eliminate another player. Note that each power can only be used once in a game
    - The seer can pick any other player and find out what their role is (one player per night)
- After the events of the night, all surviving players discuss amongst themselves and vote on a player to eliminate
- Werewolves win if their numbers equal or exceed Villagers.
- Villagers win if all Werewolves are eliminated.

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv

from typing import Optional, Union, Sequence, Any, List
from functools import partial
import openai  
import faiss
import numpy as np
import json
import csv
from datetime import datetime
from zoneinfo import ZoneInfo

import numpy as np
from loguru import logger

from agentscope.parsers.json_object_parser import MarkdownJsonDictParser
from agentscope.parsers import ParserBase
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.agents import AgentBase
from agentscope.memory.temporary_memory import TemporaryMemory
from agentscope.pipelines.functional import sequentialpipeline
import agentscope

from utils.werewolf_utils import (
    extract_name_and_id,
    n2s,
    set_parsers,
)

## Vector Store Implementations
Vector store classes 

In [5]:
class ReflectiveVectorstoreMemory:
    """Reflective Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)
        self.summaries = []  # To store summaries

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = client.embeddings.create(
            input=text,
            model=self.embedding_model
        )
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        for msg in message:
            self.messages.append(msg.name + ": " + msg.content)
        
    def summarize_cycle(self, cycle_type: str = "day"):
        """
        Generate a summary of the conversation after a day/night cycle,
        and add it to the vector store.
        
        Args:
            cycle_type (str): The type of cycle ("day" or "night").
        """
        client = openai.OpenAI()
        
        # Combine all messages since the last summary
        history = "\n".join(self.messages)
        
        # Use the slow mind (gpt-4o) to summarize
        slow_mind_response = client.chat.completions.create(
            model="gpt-4o",  # Slow mind model
            messages=[
                {"role": "system", "content": f"Summarize the following chat history from the last {cycle_type}:"},
                {"role": "user", "content": history}, 
            ],
        )
        summary = slow_mind_response.choices[0].message.content

        # Embed the summary and add it to the vector store
        embedding = self._get_embedding(summary)
        self.index.add(np.array([embedding]))
        self.summaries.append(summary)

        # Clear messages for the next cycle (current implementation gets summary from the point after the last summary was created)
        self.messages.clear()

    def get_relevant_summaries(self, query: str, top_k: int = 1):
        """Retrieve the top-k most relevant summaries.
        
        Args:
            query (str): query to find similar summaries to.
            top_k (int): number of relevant summaries to retrieve.
        """
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)
        results = [
            self.summaries[idx] for idx in indices[0] if idx < len(self.summaries)
        ]
        return results
    
    
class VectorstoreMemory:
    """Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = client.embeddings.create(
            input=text,
            model=self.embedding_model
        )
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        # add embedding of message to vector store
        embeddings = [self._get_embedding(msg.name + ": " + msg.content) for msg in message]
        self.index.add(np.array(embeddings)) 
        self.messages.extend(message)

    def get_relevant_messages(self, query: str, top_k: int = 10):
        """Retrieve the top-k most relevant messages."""
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)
        results = [
            self.messages[idx] for idx in indices[0] if idx < len(self.messages)
        ]
        return results

## Custom Agents (edit here)
You can define custom Agents by inheriting from the AgentBase class like shown here.

Below is a copy of the DictDialogAgent that generates responses in a dict format that is compatible with our simulator. More documentation on AgentScope agents can be found here https://doc.agentscope.io/build_tutorial/builtin_agent.html, existing agent implementations can be found here https://github.com/modelscope/agentscope/tree/main/src/agentscope/agents.

For our case, modifying the agent class is, in conjunction with the parsers we pass in (more details in next cell), play a critical role in defining our agent behavior. We can edit this section by either defining a brand new agent type we want to explore, or modifying the current one. The primary (and mostly only) source of focus should be the reply the function in the agent class as that controls what an agent uses to generate a response, particular when we are working with different architectures, eg. how we use memory, single vs multi agent, we make those edits here.

In [6]:
class Lawyer(AgentBase):
    """An agent specializing in constructing LLM-based arguments to eliminate a specific target in the Werewolf game."""

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        parser,
        context_messages: list,
        memory
    ) -> None:
        """
        Initialize the Lawyer agent.

        Args:
            name (str):
                Name of the Lawyer agent.

            sys_prompt (str):
                System prompt containing role instructions and context.

            model_config_name (str):
                Identifier for the model configuration that determines the LLM to be used.

            parser (ParserBase):
                Object responsible for formatting LLM outputs and extracting structured responses.

            context_messages (list):
                A list of prior messages providing context for the lawyer’s argumentation.

            memory:
                Memory management component used to store conversation history or additional data.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )
        self.parser = parser
        self.context_messages = context_messages
        self.memory = memory

    def argue(self, game_state: dict, available_targets: list) -> dict:
        """
        Generate a rationale for eliminating a particular player in the Werewolf game.

        Args:
            game_state (dict):
                The current state of the game (e.g., who is alive, who is dead).

            available_targets (list):
                List of potential targets that can be selected for elimination.

        Returns:
            dict:
                A dictionary containing:
                  - "target" (str): The chosen player to eliminate.
                  - "argument" (str): The Lawyer’s supporting reasoning.
        """
        if not available_targets:
            return {"target": None, "argument": "No valid targets available."}

        # Construct prompt for argument generation
        lawyer_prompt = f"""
        You are a lawyer in a game of Werewolf. Argue for eliminating one of {', '.join(available_targets)}.
        
        Consider:
        - The player's past behavior (e.g., suspicions, accusations, voting history).
        - Logical reasoning that would convince the judge.
        """

        # Prepare the prompt for the model
        formatted_prompt = self.model.format(
            Msg("system", lawyer_prompt, role="system"),
            self.context_messages,
            Msg("system", self.parser.format_instruction, "system"),
        )

        # Call the LLM to generate an argument
        response = self.model(formatted_prompt)

        return {
            "target": extract_name_and_id(response.text)[0],  # Extract chosen target
            "argument": response.text  # Return generated argument
        }


class Judge(AgentBase):
    """An agent that evaluates competing arguments and selects who should be eliminated in the Werewolf game."""

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        parser,
        context_messages: list,
        memory
    ) -> None:
        """
        Initialize the Judge agent.

        Args:
            name (str):
                Name of the Judge agent.

            sys_prompt (str):
                System prompt containing role instructions and context.

            model_config_name (str):
                Identifier for the model configuration that determines the LLM to be used.

            parser (ParserBase):
                Object responsible for formatting LLM outputs and extracting structured responses.

            context_messages (list):
                List of prior messages providing context for the Judge’s decision-making.

            memory:
                Memory component used to store and retrieve conversation or historical data.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )
        self.parser = parser
        self.context_messages = context_messages
        self.memory = memory

    def decide(self, game_state: dict, argument_1: dict, argument_2: dict):
        """
        Evaluate two lawyers' arguments and choose a target for elimination.

        Args:
            game_state (dict):
                The current state of the Werewolf game (e.g., which players are alive or dead).

            argument_1 (dict):
                First lawyer’s argument dictionary with:
                  - "target" (str): Proposed player to eliminate.
                  - "argument" (str): Reasoning for that choice.

            argument_2 (dict):
                Second lawyer’s argument dictionary with the same keys as argument_1.

        Returns:
            The raw response from the model, containing the Judge’s final decision.
        """
        argument_1_target = argument_1["target"]
        argument_1_argument = argument_1["argument"]
        
        argument_2_target = argument_2["target"]
        argument_2_argument = argument_2["argument"]

        # Construct prompt for decision-making
        judge_prompt = f"""
        You are a judge in a game of Werewolf, responsible for deciding who should be eliminated.
        
        Game State:
        - Alive players: {game_state["survivors"]}
        - Dead players: {game_state["dead"]}
        - Known werewolves: {game_state["werewolves"]}

        The two lawyers have made their arguments:
        - **argument_1** wants to eliminate {argument_1_target}: "{argument_1_argument}"
        - **argument_2** wants to eliminate {argument_2_target}: "{argument_2_argument}"

        Evaluate both arguments carefully and decide who should be eliminated.
        Your decision should consider:
        - Which argument is more logically sound.
        - The strategic advantage of eliminating one target over the other.
        - Any potential risks for the werewolves.
        """

        # Prepare the prompt for the model
        formatted_prompt = self.model.format(
            Msg("system", judge_prompt, role="system"),
            self.context_messages,
            Msg("system", self.parser.format_instruction, "system"),
        )

        raw_response = self.model(formatted_prompt)

        return raw_response


class WerewolfAgent(AgentBase):
    """
    Represents a "werewolf" in the game, producing responses in dict form.

    Optionally uses reflective memory before retrieving context from a FAISS vectorstore.
    Contains parsing capabilities to ensure structured outputs.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        reflect_before_vectorstore: bool,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
    ) -> None:
        """
        Initialize the WerewolfAgent.

        Args:
            name (str):
                Name of the agent.

            sys_prompt (str):
                System prompt providing role context and instructions.

            model_config_name (str):
                Name of the model configuration, indicating which LLM will be used.

            reflect_before_vectorstore (bool):
                Whether to summarize or reflect on memory before retrieving vectorstore data.

            similarity_top_k (int, optional):
                The number of most similar messages/summaries to retrieve from the vectorstore.

            openai_api_key (str, optional):
                API key for OpenAI integration.

            model:
                The underlying LLM for generating responses.

            parser (ParserBase):
                Parser responsible for structuring, filtering, and formatting model outputs.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.similarity_top_k = similarity_top_k
        self.model_config_name = model_config_name

        # Set OpenAI API key
        openai.api_key = openai_api_key

        # Initialize FAISS-based memory store
        if reflect_before_vectorstore:
            self.memory = ReflectiveVectorstoreMemory()
        else:
            self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """
        Set the parser for handling model outputs.

        This parser governs how responses are formatted, parsed,
        and stored (including any filtering of fields).
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Generate a response based on input messages, optionally retrieving context from memory.

        Args:
            x (Optional[Union[Msg, Sequence[Msg]]], optional):
                The input message(s) to process.

        Returns:
            Msg:
                The assistant's structured output, including any metadata defined by the parser.
        """
        global game_log

        # Add the input message to memory
        if x is not None:
            self.memory.add_message(message=x)
            
        query = x.content if x is not None else ""

        # Retrieve relevant messages from memory
        if self.reflect_before_vectorstore:
            if len(self.memory.summaries) > 0:
                relevant_summaries = self.memory.get_relevant_summaries(query=query, top_k=self.similarity_top_k)
                summary_context = "\n".join(relevant_summaries)
            else:
                summary_context = ""

            game_log.append(f"Summary context: {summary_context}")

            # Prepare prompt with context from retrieved summaries
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                Msg(name="system", role="system", content=f"Summary of relevant past conversations: {summary_context}")
                or x,  # type: ignore[arg-type]
                Msg("system", self.parser.format_instruction, "system"),
            )
        else:
            relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k) or []

            game_log.append(f"Relevant messages: {'\n'.join([msg.content for msg in relevant_messages])}")

            # Prepare prompt with retrieved messages similar to input message
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                relevant_messages
                or x,  # type: ignore[arg-type]
                Msg("system", self.parser.format_instruction, "system"),
            )

        # Call the LLM
        raw_response = self.model(prompt)

        self.speak(raw_response.stream or raw_response.text)

        # Parse the raw response
        res = self.parser.parse(raw_response)

        msg = Msg(
            self.name,
            content=self.parser.to_content(res.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(res.parsed),
        )

        # Save the response to memory
        self.memory.add_message(message=msg)

        return msg

    def lawyer_judge_decision(self, game_state: dict, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Make an elimination decision by invoking two Lawyers and one Judge.

        Args:
            game_state (dict):
                The current state of the Werewolf game (e.g., active players, werewolves).

            x (Optional[Union[Msg, Sequence[Msg]]], optional):
                Optionally, messages to be recorded in memory before decision-making.

        Returns:
            Msg:
                The final decision from the Judge as a structured message.
        """
        global game_log

        # Add input message to memory
        if x is not None:
            self.memory.add_message(message=x)
            
        query = x.content if x is not None else ""

        # Retrieve relevant messages from memory (summaries or direct messages)
        if self.reflect_before_vectorstore:
            if len(self.memory.summaries) > 0:
                relevant_summary = self.memory.get_relevant_summaries(query=query, top_k=self.similarity_top_k)
                summary_context = "\n".join(relevant_summary)
            else:
                summary_context = ""
            
            relevant_messages = [Msg(name="system", role="system", content=f"Summary of relevant past conversations: {summary_context}")]
            game_log.append(f"Summary context: {summary_context}")

            # Prepare prompt with context from retrieved summaries
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                Msg(name="system", role="system", content=f"Summary of relevant past conversations: {summary_context}")
                or x,  # type: ignore[arg-type]
                Msg("system", self.parser.format_instruction, "system"),
            )
        else:
            relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

            game_log.append(f"Relevant messages: {'\n'.join([msg.content for msg in relevant_messages])}")

            # Prepare prompt using retrieved messages
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                relevant_messages
                or x,  # type: ignore[arg-type]
                Msg("system", self.parser.format_instruction, "system"),
            )
            
        # Initialize the lawyers and judge
        lawyer_1 = Lawyer(
            f'{self.name} (Lawyer 1)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,              
            self.memory
        )

        lawyer_2 = Lawyer(
            f'{self.name} (Lawyer 2)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,
            self.memory
        )

        judge = Judge(
            f'{self.name} (Judge)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,
            self.memory
        )
    
        available_targets = [player for player in game_state["survivors"] if player not in game_state["werewolves"]]

        lawyer_1_argument = lawyer_1.argue(game_state, available_targets)
        available_targets.remove(lawyer_1_argument["target"])
        lawyer_2_argument = lawyer_2.argue(game_state, available_targets)

        raw_response = judge.decide(game_state, lawyer_1_argument, lawyer_2_argument)

        game_log.append(f"Lawyer 1 argument: {lawyer_1_argument}")
        game_log.append(f"Lawyer 2 argument: {lawyer_2_argument}")
        game_log.append(f"Judge decision: {raw_response.text}")

        self.speak(raw_response.stream or raw_response.text)

        # Parse response
        res = self.parser.parse(raw_response)

        msg = Msg(
            self.name,
            content=self.parser.to_content(res.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(res.parsed),
        )

        # Store final decision
        self.memory.add_message(message=msg)

        return msg

    def observe(self, x: Union[Msg, Sequence[Msg]]) -> None:
        """
        Record incoming messages in memory without generating a reply.

        Args:
            x (Union[Msg, Sequence[Msg]]):
                The message(s) to be stored in memory for future context.
        """
        if x is not None:
            self.memory.add_message(message=x)

    def summarize_cycle(self, cycle_type: str = "day"):
        """
        Produce a summary of key events at the end of a day or night cycle.

        Args:
            cycle_type (str, optional):
                The type of cycle being summarized, either "day" or "night".
        """
        self.memory.summarize_cycle(cycle_type=cycle_type)


class NormalAgent(AgentBase):
    """
    A general-purpose agent that returns structured responses in JSON/dict format.

    Integrates with a FAISS vectorstore for context retrieval.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        reflect_before_vectorstore: bool,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
    ) -> None:
        """
        Initialize the NormalAgent.

        Args:
            name (str):
                Agent name.

            sys_prompt (str):
                The system prompt containing the agent’s role instructions.

            model_config_name (str):
                Indicates which model configuration the agent should use.

            reflect_before_vectorstore (bool):
                Flag to determine if reflective memory is used before vectorstore retrieval.

            similarity_top_k (int, optional):
                The number of top similar items from the vectorstore to include as context.

            openai_api_key (str, optional):
                API key for integrating with OpenAI services.

            model:
                The LLM used to generate responses.

            parser (ParserBase):
                Responsible for formatting and parsing outputs, as well as filtering fields.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        openai.api_key = openai_api_key
        self.similarity_top_k = similarity_top_k
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """
        Configure the parser that dictates how outputs are formatted, parsed, and stored.

        By adjusting the parser, you can control the entire pipeline of formatting and extraction
        for the model's responses.
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Generate a response based on provided input messages, using stored context as needed.

        Args:
            x (Optional[Union[Msg, Sequence[Msg]]], optional):
                The incoming user or system messages.

        Returns:
            Msg:
                A structured message including the assistant's content and any parsed metadata.
        """
        global game_log

        # Store incoming message(s) in memory
        if x is not None:
            self.memory.add_message(message=x)
            
        query = x.content if x is not None else ""

        # Retrieve similar messages from memory
        relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

        game_log.append(f"Relevant messages from vectorstore: {'\n'.join([msg.content for msg in relevant_messages])}")

        # Prepare the prompt with relevant context
        prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            relevant_messages
            or x,  # type: ignore[arg-type]
            Msg("system", self.parser.format_instruction, "system"),
        )

        # Call the LLM
        raw_response = self.model(prompt)

        self.speak(raw_response.stream or raw_response.text)

        # Parse the model's output
        res = self.parser.parse(raw_response)

        msg = Msg(
            self.name,
            content=self.parser.to_content(res.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(res.parsed),
        )
        
        # Save the model's response to memory
        self.memory.add_message(message=msg)

        return msg

    def observe(self, x: Union[Msg, Sequence[Msg]]) -> None:
        """
        Record messages in memory without generating a response.

        Args:
            x (Union[Msg, Sequence[Msg]]):
                One or more messages to store for future reference.
        """
        if x is not None:
            self.memory.add_message(message=x)


# Register your custom classes
agentscope.agents.NormalAgent = NormalAgent
agentscope.agents.WerewolfAgent = WerewolfAgent
agentscope.agents.Lawyer = Lawyer
agentscope.agents.Judge = Judge

## Prompts and Parser (edit minimally)
Below are the prompts that control and run the Werwolf game. They are implemented using the built in AgentScope MarkdownJsonDictParser which allows us to generate responses in a dictionary format that is compatible with our game. Currently structured to obtain both the private thoughts of an Agent which remain to themselves and showcase their reasoning, and the words that they speak to other agents.

Since these prompts control the game logic itself, we should try to limit the prompt edits here to a minimum, but we can make edits to the parser to control the reasoning process a particular role goes through before making a decision, eg. instead of a thought field, we ask it to make ask it to provide a reason to vote for a particular agent, and a reason to not vote for a particular agent, then ask it to make a final decision based off those fields. Fields other than "thought" however should not be touched as they play a critical role to the control flow of the game.

More details on the AgentScope parsers can be found here https://doc.agentscope.io/build_tutorial/structured_output.html

In [7]:
class Prompts:
    """Prompts for werewolf game"""

    to_wolves = (
        "{}, if you are the only werewolf, eliminate a player. Otherwise, "
        "discuss with your teammates and reach an agreement."
    )

    wolves_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "finish_discussion": "whether the discussion reached an agreement or not (true/false)",
        },
        required_keys=["thought", "speak", "finish_discussion"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["finish_discussion"],
    )

    to_wolves_vote = "Which player do you vote to kill?"

    wolves_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
        },
        required_keys=["thought", "vote"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're the witch. Tonight {dead_name} is eliminated. "
        "Would you like to resurrect {dead_name}?"
    )

    to_witch_resurrect_no = "The witch has chosen not to resurrect the player."
    to_witch_resurrect_yes = "The witch has chosen to resurrect the player."

    witch_resurrect_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "whether to resurrect the player and the reason",
            "resurrect": "whether to resurrect the player or not (true/false)",
        },
        required_keys=["thought", "speak", "resurrect"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["resurrect"],
    )

    to_witch_poison = (
        "Would you like to eliminate one player? If yes, "
        "specify the player_name."
    )

    witch_poison_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "eliminate": "whether to eliminate a player or not (true/false)",
        },
        required_keys=["thought", "speak", "eliminate"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["eliminate"],
    )

    to_seer = (
        "{}, you're the seer. Which player in {} would you like to check "
        "tonight?"
    )

    seer_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "player_name",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    to_seer_result = "Okay, the role of {} is a {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the situation and the information you gain, to vote a "
        "player eliminated among alive players and to win the game, what do "
        "you want to say to others? You can decide whether to reveal your "
        "role."
    )

    survivors_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    survivors_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
        },
        required_keys=["thought", "vote"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_all_vote = (
        "Given the game rules and your role, based on the situation and the"
        " information you gain, to win the game, it's time to vote one player"
        " eliminated among the alive players. Which player do you vote to "
        "kill?"
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

    
# Moderator message function
HostMsg = partial(Msg, name="Moderator", role="assistant", echo=True)

## Game Initialization (edit here)
To initialize the agents, you must define their parameters and settings in the config objects that are passed in for initialization. There is a model config, which defines the base foundational model being used, and an agent config, which defines each of the agents being used in the game, including which model their using, their name, and what type of Agent they are (based off the agent classes we defined earlier). 

Pay particular attention to the system prompt, this is what defines the rules of the game to the agent and gives them the role and what their responsibilities are, we could perhaps do some prompt engineering with that.

Also we can play around with the settings of the game, eg. max rounds, how many werewolves we have, etc. Just make sure to update the roles, witch, seer objects below accordingly.

In [8]:
# Holds game data 
game_log = []

# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")


## Utility Functions (don't edit)
Functions to check and update game state throughout

In [9]:
def check_winning(game_state: dict, alive_agents: list, wolf_agents: list, host: str) -> bool:
    """check which group wins"""
    if len(wolf_agents) * 2 >= len(alive_agents):
        msg = Msg(host, Prompts.to_all_wolf_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "werewolves"
        logger.chat(msg)
        return True
    if alive_agents and not wolf_agents:
        msg = Msg(host, Prompts.to_all_village_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "villagers"
        logger.chat(msg)
        return True
    return False

def update_alive_players(
    game_state: dict,
    survivors: Sequence[AgentBase],
    wolves: Sequence[AgentBase],
    dead_names: Union[str, list[str]],
) -> tuple[list, list]:
    """update the list of alive agents"""
    if not isinstance(dead_names, list):
        dead_names = [dead_names]
    for player in dead_names:
        for player in dead_names:
            if player in game_state["survivors"]:
                game_state["survivors"].remove(player)

        if player not in game_state["dead"]:
            game_state["dead"].append(player)  
        else:
            continue
    return [_ for _ in survivors if _.name not in dead_names], [
        _ for _ in wolves if _.name not in dead_names
    ]


def majority_vote(votes: list) -> Any:
    """majority_vote function"""
    votes_valid = [item for item in votes if item != "Abstain"]
    # Count the votes excluding abstentions.
    unit, counts = np.unique(votes_valid, return_counts=True)
    return unit[np.argmax(counts)]

def save_logs_to_logfile(logs: list, filepath: str):
    """ Saves a list of log messages to a .log file, each on a new line. """
    # Ensure the directory exists before writing
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    # Write each log message to a new line in the .log file
    with open(filepath, mode="w", encoding="utf-8") as file:
        file.writelines(log + "\n" for log in logs)

    print(f"Game log saved to {filepath}")

def save_results(win_rate_filepath: str, row: dict):
    """
    Appends a row to the CSV file containing win rates. If the file does not exist, it creates one with headers.
    
    Arguments:
    - win_rate_filepath (str): The path to the CSV file.
    - row (dict): A dictionary containing "raw_log_filepath" and "custom_agent_won".
    """

    # Check if the file exists
    file_exists = os.path.isfile(win_rate_filepath)
    
    # Open file in append mode
    with open(win_rate_filepath, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["raw_log_filepath", "custom_agent_won"])
        
        # If file does not exist, write the header first
        if not file_exists:
            writer.writeheader()
        
        # Write the row to the CSV file
        writer.writerow(row)

    print(f"Row added to {win_rate_filepath}: {row}")

def generate_log_filepath(basepath: str, game_num: int):
    # Get current time in EST
    est_now = datetime.now(ZoneInfo("America/New_York"))

    # Format as ISO 8601 with dashes
    iso_timestamp = est_now.isoformat(timespec='seconds').replace(":", "-")

    return f"{basepath}_{game_num}_{iso_timestamp}.log"

def custom_agent_won(game_state: dict, alive_agents: list, wolf_agents: list):
    """Records whether custom agent won (even if game didn't complete)"""

    # If game is completed, then wolf wins if it is still alive
    if game_state["endgame"]:
        return len(wolf_agents) * 2 >= len(alive_agents)
    
    # If game not completed, then custom agent wins because it is still alive and eliminated another player
    return True

## Running a Game (don't edit)
Following is a function that game through the various Night and Day phases, taking different actions for each agent based on their roles. Multi agent functionality and communication is facilitated through the AgentScope Pipeline and MsgHub, more detailed documentation found here https://doc.agentscope.io/build_api/agentscope.pipelines.pipeline.html#module-agentscope.pipelines.pipeline and https://doc.agentscope.io/build_api/agentscope.msghub.html#module-agentscope.msghub

In [10]:
def run_game(
        game_num,
        max_days_per_game,
        raw_log_filepath,
        win_rate_filepath,
        reflect_before_vectorstore,
        max_werewolf_discussion_round,
        wolves,
        seer,
        witch,
        roles,
        survivors,
        game_state
    ):
    global game_log
    game_log = []  # Store data for CSV

    for _ in range(1, max_days_per_game + 1):
        # Werewolves discussion & voting
        hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
        with msghub(wolves, announcement=hint) as hub:
            game_log.append(f"Moderator: {hint.content}")
            set_parsers(wolves, Prompts.wolves_discuss_parser)

            for _ in range(max_werewolf_discussion_round):
                x = sequentialpipeline(wolves)
                game_log.append(f"Werewolves Discussion: {x.content}")
                if x.metadata.get("finish_discussion", False):
                    break
            # Werewolves vote
            set_parsers(wolves, Prompts.wolves_vote_parser)
            hint = HostMsg(content=Prompts.to_wolves_vote)
            game_log.append(f"Moderator: {hint.content}")
            votes = [extract_name_and_id(wolf.lawyer_judge_decision(game_state, hint).content)[0] for wolf in wolves]
            dead_player = [majority_vote(votes)]
            game_log.append(f"Werewolves voted to eliminate: {dead_player[0]}")
            hub.broadcast(HostMsg(content=Prompts.to_wolves_res.format(dead_player[0])))

        # witch decision night   
        healing_used_tonight = False
        if witch in survivors:
            if not game_state["witch_healing_used"]:
                hint = HostMsg(
                    content=Prompts.to_witch_resurrect.format_map(
                        {
                            "witch_name": witch.name,
                            "dead_name": dead_player[0],
                        },
                    ),
                )
                game_log.append(f"Moderator: {hint.content}")
                set_parsers(witch, Prompts.witch_resurrect_parser)
                
                # Capture the witch's resurrection response and log it
                resurrection_response = witch(hint)
                game_log.append(f"Witch resurrection response: {resurrection_response.content}")

                if resurrection_response.metadata.get("resurrect", False):
                    healing_used_tonight = True
                    dead_player.pop()
                    game_state["witch_healing_used"] = True
                    game_log.append("Moderator: The witch has chosen to resurrect the player.")
                    HostMsg(content=Prompts.to_witch_resurrect_yes)
                else:
                    game_log.append("Moderator: The witch has chosen NOT to resurrect the player.")
                    HostMsg(content=Prompts.to_witch_resurrect_no)

            if not game_state["witch_poison_used"] and not healing_used_tonight:
                set_parsers(witch, Prompts.witch_poison_parser)
                
                # Capture the witch's poison response and log it
                poison_response = witch(HostMsg(content=Prompts.to_witch_poison))
                game_log.append(f"Witch poison response: {poison_response.content}")

                if poison_response.metadata.get("eliminate", False):
                    target_player = extract_name_and_id(poison_response.content)[0]
                    dead_player.append(target_player)
                    game_state["witch_poison_used"] = True
                    game_log.append(f"Moderator: The witch has chosen to poison {target_player}.")


        # Seer decision night
        if seer in survivors:
            hint = HostMsg(content=Prompts.to_seer.format(seer.name, n2s(survivors)))
            game_log.append(f"Moderator: {hint.content}")
            set_parsers(seer, Prompts.seer_parser)

            x = seer(hint)
            game_log.append(f"Seer Decision: {x.content}")

            role = "werewolf" if roles[extract_name_and_id(x.content)[1]] == "werewolf" else "villager"
            game_log.append(f"Moderator: The role of {extract_name_and_id(x.content)[0]} is {role}.")

        # Update survivors after night
        survivors, wolves = update_alive_players(game_state, survivors, wolves, dead_player)

        if check_winning(game_state, survivors, wolves, "Moderator"):
            game_log.append("Moderator: The game is over.")
            game_state["endgame"] = True
            break

        # update vectorstore summaries with summaries of the night
        if reflect_before_vectorstore:
            for wolf in wolves:
                wolf.summarize_cycle(cycle_type="night")

        # Daytime discussion & voting
        content = Prompts.to_all_danger.format(n2s(dead_player)) if dead_player else Prompts.to_all_peace
        game_log.append(f"Moderator: {content}")

        hints = [
            HostMsg(content=content),
            HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors))),
        ]

        with msghub(survivors, announcement=hints) as hub:
            set_parsers(survivors, Prompts.survivors_discuss_parser)
            discussion = sequentialpipeline(survivors)
            game_log.append(f"Survivors Discussion: {discussion.content}")

            # Voting
            set_parsers(survivors, Prompts.survivors_vote_parser)
            hint = HostMsg(content=Prompts.to_all_vote.format(n2s(survivors)))
            game_log.append(f"Moderator: {hint.content}")

            votes = [extract_name_and_id(_.reply(hint).content)[0] for _ in survivors]
            vote_res = majority_vote(votes)
            game_log.append(f"Daytime Vote Result: {vote_res}")

            hub.broadcast(HostMsg(content=Prompts.to_all_res.format(vote_res)))

            survivors, wolves = update_alive_players(game_state, survivors, wolves, vote_res)

            if check_winning(game_state, survivors, wolves, "Moderator"):
                game_log.append("Moderator: The game is over.")
                game_state["endgame"] = True
                break

            # update vectorstore summaries with summaries of the day
            if reflect_before_vectorstore:
                for wolf in wolves:
                    wolf.summarize_cycle(cycle_type="day")

            hub.broadcast(HostMsg(content=Prompts.to_all_continue))

    updated_raw_log_filepath = generate_log_filepath(raw_log_filepath, game_num)

    # Record raw logs
    save_logs_to_logfile(game_log, updated_raw_log_filepath)

    row = {
        "raw_log_filepath": updated_raw_log_filepath,
        "custom_agent_won": custom_agent_won(game_state, survivors, wolves)
    }

    # Record results
    save_results(win_rate_filepath, row)


In [11]:
def run_experiment(
    open_ai_key,
    raw_log_filepath,
    win_rate_filepath,
    max_days_per_game=10,
    num_games=10,
    reflect_before_vectorstore=True,
    primary_model="gpt-4o-mini",
    max_werewolf_discussion_round=3,
    similarity_top_k=1
):  
    # GAME INITIALIZATION
    
    # define model configs and settings
    model_configs = [
        {
            "model_type": "openai_chat",
            "config_name": primary_model,
            "model_name": primary_model,
            "api_key": open_ai_key,
            "generate_args": {
                "temperature": 0.5
            }
        }
    ]

    # define the config settings for each agent involved
    agent_configs = [
        {
            "class": "WerewolfAgent",
            "args": {
                "name": "Player1",
                "sys_prompt": "Act as a player in a werewolf game. You are Player1 and\nthere are totally 4 players, named Player1, Player2, Player3, Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing werewolf in this game.\n",
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player2",
                "sys_prompt": "Act as a player in a werewolf game. You are Player2 and\nthere are totally 4 players, named Player1, Player2, Player3, Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing villager in this game.\n",
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player3",
                "sys_prompt": "Act as a player in a werewolf game. You are Player3 and\nthere are totally 4 players, named Player1, Player2, Player3, Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing seer in this game.\n",
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player4",
                "sys_prompt": "Act as a player in a werewolf game. You are Player4 and\nthere are totally 4 players, named Player1, Player2, Player3, Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing witch in this game.\n",
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": 1,
                "openai_api_key": open_ai_key
            }
        }
    ]

    # RUNNING GAME

    # read model and agent configs, and initialize agents automatically
    survivors = agentscope.init(
        model_configs=model_configs,
        agent_configs=agent_configs,
        project="Werewolf",
    )

    # update these accordingly based off agent configs
    roles = ["werewolf", "villager", "seer", "witch"]
    wolves, witch, seer = [survivors[0]], survivors[-1], survivors[-2]

    # initialize game state
    game_state = {
        "werewolves": [player.name for player in wolves],
        "villagers": [player.name for player in survivors if player not in wolves],
        "seer": [seer.name],
        "witch": [witch.name],
        "survivors": [player.name for player in survivors],
        "dead": [],
        "witch_healing_used": False,
        "witch_poison_used": False,
        "endgame": False,
        "winner": None
        
    }

    for game_num in range(1, num_games + 1):
        run_game(
            game_num,
            max_days_per_game,
            raw_log_filepath,
            win_rate_filepath,
            reflect_before_vectorstore,
            max_werewolf_discussion_round,
            wolves,
            seer,
            witch,
            roles,
            survivors,
            game_state
        )

In [ ]:
run_experiment(
    open_ai_key=openai_key,
    raw_log_filepath="courtroom_logs_reflection/courtroom_raw",
    reflect_before_vectorstore=True,
    win_rate_filepath="courtroom_win_rate_reflection.txt",
    max_days_per_game=2,
    num_games=1,
    similarity_top_k=2
)

2025-02-10 23:16:14 | INFO     | agentscope.manager._model:load_model_configs:115 - Load configs for model wrapper: gpt-4o-mini
2025-02-10 23:16:14 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-10 23:16:14 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o-mini: 'Model [gpt-4o-mini] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-10 23:16:14 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-10 23:16:14 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o-mini: 'Model [gpt-4o-mini] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-10 23:16:14 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-10 23:16:14 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_len